In [1]:
!pip install dataretrieval

In [ ]:


# !pip install netCDF4
import xarray as xr
# from netCDF4 import Dataset
import numpy as np
import pandas as pd
import geopandas as gpd
# from google.colab import drive
from matplotlib import pyplot as plt
import datetime

import glob
from dataretrieval import nwis as nwis



ModuleNotFoundError: No module named 'google.colab'

In [ ]:
cb_centerlines = gpd.read_file('/content/drive/MyDrive/river_tinder_assets/centerlines/s2_platte_centerlines_4326.shp').to_crs(3857)
merit_centerlines = gpd.read_file('/content/drive/MyDrive/MERIT/riv_pfaf_7_MERIT_Hydro_v07_Basins_v01_bugfix1.shp').to_crs(3857)

In [ ]:
Qarray = xr.open_dataset('/content/drive/MyDrive/grades_Q/GRADES-hydroDL_V2.0_pfaf_07_2024.nc')

In [4]:
rwc_dfs = []

files = glob.glob('/content/drive/MyDrive/all_rwc_exports/*.csv')

for f in files:
    rwc_dfs.append(pd.read_csv(f))

rwc_df = pd.concat(rwc_dfs)

In [5]:
rwc_gdf = gpd.GeoDataFrame(rwc_df, geometry=gpd.points_from_xy(rwc_df.x, rwc_df.y, crs=4326))
rwc_gdf.scene_date = pd.to_datetime(rwc_gdf.scene_date, format='ISO8601')

In [ ]:
TDF = rwc_gdf.loc[rwc_gdf.scene_date <= datetime.datetime(2018, 12, 31)]

In [ ]:
mwidths = TDF.groupby('geometry')['width'].agg('mean').reset_index()

In [ ]:
mwidths = mwidths.loc[mwidths.width > 0]

In [ ]:
mwidths['logw'] = np.log(mwidths.width)

In [ ]:
mwidths = gpd.GeoDataFrame(mwidths, geometry='geometry')
mwidths.plot(column='logw')

In [6]:
gage_list = ['06764880', '06775900', '06775500', '06770200', '06767970', '06768000',
             '06784000', '06781900', '06785000', '06790500', '06800500', '06793000',
             '06800000', '06777495', '06786000', '06799350', '06799315', '06799000',
             '06797500', '06785500', '06710247', '06711565', '06714215', '06721000',
             '06759500', '06620000', '06680500', '06630000', '06674500', '06635000',
             '06670500', '06657000', '06652000']




In [7]:
gages = nwis.get_info(sites=gage_list)[0][['site_no', 'geometry']]
gages

,site_no,geometry
0,06620000,POINT (-106.33919 40.93664)
1,06630000,POINT (-107.0576 41.87217)
2,06635000,POINT (-106.51314 42.00967)
3,06652000,POINT (-105.15921 42.65248)
4,06657000,POINT (-104.62802 42.24101)
5,06670500,POINT (-104.53835 42.20051)
6,06674500,POINT (-104.05334 41.98857)
7,06680500,POINT (-103.67556 41.84994)
8,06710247,POINT (-105.01498 39.63249)
9,06711565,POINT (-105.00415 39.66499)


In [9]:
gage_buffers = gages.to_crs(3857)
# gages = gages.rename({'geometry': 'point'})
buffgeoms = gages.apply(lambda x: x['geometry'].buffer(1000), axis=1)
gage_buffers = gage_buffers.set_geometry(buffgeoms)
gage_buffers

,site_no,geometry
0,06620000,"POLYGON ((893.661 40.937, 888.846 -57.081, 874..."
1,06630000,"POLYGON ((892.942 41.872, 888.127 -56.145, 873..."
2,06635000,"POLYGON ((893.487 42.01, 888.672 -56.007, 874...."
3,06652000,"POLYGON ((894.841 42.652, 890.026 -55.365, 875..."
4,06657000,"POLYGON ((895.372 42.241, 890.557 -55.776, 876..."
5,06670500,"POLYGON ((895.462 42.201, 890.646 -55.817, 876..."
6,06674500,"POLYGON ((895.947 41.989, 891.131 -56.029, 876..."
7,06680500,"POLYGON ((896.324 41.85, 891.509 -56.167, 877...."
8,06710247,"POLYGON ((894.985 39.632, 890.17 -58.385, 875...."
9,06711565,"POLYGON ((894.996 39.665, 890.181 -58.352, 875..."


In [10]:
rwc_points = rwc_gdf.geometry.unique()
n_geoms = len(rwc_points)
gindex = range(n_geoms)
geom_gindex = gpd.GeoDataFrame({'geometry': rwc_points, 'gindex': gindex})

rwc_gdf = pd.merge(rwc_gdf, geom_gindex, how='left', on='geometry')

In [11]:
# geom_gindex = geom_gindex.to_crs(3857)
gage_rwc_points = geom_gindex.sjoin(gage_buffers, predicate='within').to_crs(4326)


/usr/local/lib/python3.12/dist-packages/geopandas/geodataframe.py:2560: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:3857

  return geopandas.sjoin(


In [ ]:
gage_rwc = pd.merge(gage_rwc_points[['gindex', 'site_no']], rwc_gdf, how='left', on='gindex')

In [ ]:
gage_rwc.columns

In [ ]:
gage_rwc.loc[(gage_rwc.cloud_mask == 0) & gage_rwc.loc[(gage_rwc.snow_mask == 0)]]

IFMHA

In [ ]:
gage_nums = [int(site_no) for site_no in gage_list]
ifmha = pd.read_csv('/content/drive/MyDrive/IFMHA/IFMHA.csv')

ifmha = ifmha.loc[ifmha.site_no.isin(gage_nums)]

ifmha.measurement_dt = pd.to_datetime(ifmha.measurement_dt)
ifmha = ifmha.loc[(ifmha.measurement_dt >= datetime.datetime(2018, 1, 1)) & (ifmha.measurement_dt <= datetime.datetime(2024, 12, 31))]

ifmha['date'] = ifmha.measurement_dt.dt.date
ifmha.chan_width = ifmha.chan_width / 3.28
ifmha.site_no = ifmha.apply(lambda x: f'0{x.site_no}', axis=1)


ifmha = ifmha[['site_no', 'date', 'chan_width']]

In [ ]:
ifmha

In [ ]:
width_dfs = []
for gage in (gage_list):
    url = f'https://api.waterdata.usgs.gov/samples-data/results/fullphyschem?mimeType=text%2Fcsv&monitoringLocationIdentifier=USGS-{gage}&characteristicGroup=Physical&characteristicUserSupplied=Width%20of%20stream'

    tempdf = pd.read_csv(url)

    ws = tempdf[['Location_Identifier', 'Activity_StartDate', 'Result_Measure']]
    width_dfs.append(tempdf)

isw = pd.concat(width_dfs)

isw['site_no'] = isw.apply(lambda x: x['Location_Identifier'].split('-')[1], axis=1)
isw = isw.rename(columns={'Activity_StartDate': 'date', 'Result_Measure': 'width_ft'})
isw['date'] = isw.apply(lambda x: datetime.date(int(x['date'].split('-')[0]), int(x['date'].split('-')[1]), int(x['date'].split('-')[2])), axis=1)
isw['chan_width'] = isw.width_ft / 3.28
isw = isw[['date', 'chan_width', 'site_no']]
isw

In [ ]:
all_isw = pd.concat([isw, ifmha])